In [1]:
import MDAnalysis as mda
import MDAnalysis.analysis.align
import pandas as pd
import BioSimSpace as bss
import nglview as nv
import numpy as np
import os
import glob
import MDAnalysis.analysis.rms

<frozen importlib._bootstrap>:228: RuntimeWarning: to-Python converter for std::vector<double, std::allocator<double> > already registered; second conversion method ignored.


In [8]:
path = "../azetidinimines/kpc2/equilibration/unbound/"
lig_1_number = "7aga"
lig_2_number = "7dga"

lig_1 = bss.IO.readMolecules(path+f"ligand_{lig_1_number}/npt/ligand_{lig_1_number}.*7")[0]
lig_2 = bss.IO.readMolecules(path+f"ligand_{lig_2_number}/npt/ligand_{lig_2_number}.*7")[0]
mapping = bss.Align.matchAtoms(lig_1, lig_2)
bss.Align.viewMapping(lig_1, lig_2, mapping)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
merged = bss.Align.merge(lig_1, lig_2, mapping)
for atom in merged.getAtoms():
    print(atom._sire_object.property("element0"), atom._sire_object.property("element1"))

Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Nitrogen (N, 7) Nitrogen (N, 7)
Carbon (C, 6) Carbon (C, 6)
Nitrogen (N, 7) Nitrogen (N, 7)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Oxygen (O, 8) Oxygen (O, 8)
Carbon (C, 6) Carbon (C, 6)
Carbon (C, 6) Carbon (C, 6)
Hydrogen (H, 1) Hydrogen (H, 1)
Hydrogen (H, 1) Hydrogen (H, 1)
Hydrogen (H, 1) Iodine (I, 53)
Hydrogen (H, 1) Hydrogen (H, 1)
Hydrogen (H, 1) Hydrogen (H, 1)
Hydrogen (H, 1) Hydrogen (H, 1)
Hydrogen (H, 1) Hydrogen (H, 1)
Hydrogen (H, 1) Hydrogen (H, 1)
Hydrogen (H, 1) Hydrogen (H, 1)
Hydro

In [2]:
ligand_path = "../kpc2/inputs/ligands/"
ligand_files = sorted(glob.glob(f"{ligand_path}*.sdf"))
protein_file = "../kpc2/inputs/protein/kpc2_tleap"

protein = bss.IO.readMolecules([protein_file + ".rst7", protein_file + ".prm7"])[0]
ligands = [bss.IO.readMolecules(ligand_file)[0] for ligand_file in ligand_files]

In [3]:
ligand_file = "../kpc2/inputs/ligands/ligand_13.sdf"
ligand = bss.IO.readMolecules(ligand_file)[0]
ligand_parameters = bss.Parameters.gaff2(ligand, net_charge=-1).getMolecule()

In [4]:
box_edges = 20
box_axis_unit = bss.Units.Length.angstrom
solvent_force_field = "tip3p"
box_min, box_max = ligand_parameters.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
box_sizes = [x + int(box_edges) * box_axis_unit for x in box_size] 
box, angles = bss.Box.cubic(max(box_sizes))
ligand_solvated = bss.Solvent.solvate(solvent_force_field, molecule=ligand_parameters, box=box, angles=angles)


In [6]:
bss.IO.saveMolecules("ligand_13", ligand_solvated, ["PRM7", "RST7"])

['/home/jguven/projects/metalloenzymes/scripts/ligand_13.prm7',
 '/home/jguven/projects/metalloenzymes/scripts/ligand_13.rst7']

In [4]:
ligand_path = "../kpc2/inputs/ligands/"
ligand_files = sorted(glob.glob(f"{ligand_path}*.sdf"))
ligands = [bss.IO.readMolecules(filepath)[0] for filepath in ligand_files]
ligand_names = [filepath.split("/")[-1].replace(".sdf","") for filepath in ligand_files]
transformations, lomap_scores = bss.Align.generateNetwork(ligands, plot_network=False, names=ligand_names, work_dir=ligand_path)

In [10]:
unbound_path = "../kpc2/equilibration/unbound/"
ligand_1_number = 11
ligand_2_number = 5
ligand_1_system = bss.IO.readMolecules([f"{unbound_path}/ligand_{ligand_1_number}/npt/ligand_{ligand_1_number}.prm7",
                                        f"{unbound_path}/ligand_{ligand_1_number}/npt/ligand_{ligand_1_number}.rst7"])
ligand_2_system = bss.IO.readMolecules([f"{unbound_path}/ligand_{ligand_2_number}/npt/ligand_{ligand_2_number}.prm7",
                                        f"{unbound_path}/ligand_{ligand_2_number}/npt/ligand_{ligand_2_number}.rst7"])
ligand_1 = ligand_1_system.getMolecule(0)
ligand_2 = ligand_2_system.getMolecule(0)
mapping = bss.Align.matchAtoms(ligand_1, ligand_2, complete_rings_only=True) 
inverse_mapping = {value:key for key,value in mapping.items()}
ligand_2_aligned = bss.Align.rmsdAlign(ligand_2, ligand_1, inverse_mapping)

merged_ligands = bss.Align.merge(ligand_1, ligand_2_aligned, mapping) 

ligand_1_system.removeMolecules(ligand_1)
ligand_1_system.addMolecules(merged_ligands)
unbound_system = ligand_1_system


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 unbound_path = "../kpc2/equilibration/unbound/"                                             │
│    2 ligand_1_number = 11                                                                        │
│    3 ligand_2_number = 5                                                                         │
│ ❱  4 ligand_1_system = bss.IO.readMolecules([f"{unbound_path}/ligand_{ligand_1_number}/npt/li    │
│    5 │   │   │   │   │   │   │   │   │   │   f"{unbound_path}/ligand_{ligand_1_number}/npt/li    │
│    6 ligand_2_system = bss.IO.readMolecules([f"{unbound_path}/ligand_{ligand_2_number}/npt/li    │
│    7 │   │   │   │   │   │   │   │   │   │   f"{unbound_path}/ligand_{ligand_2_number}/npt/li    │
│                                                                                                  │
│ /home/jguven/Software/miniconda3/envs/obss/lib/python3.10/site-packages/BioSimSpace/IO/_io.py:44 │
│ 9 in readMolecules                                                                               │
│                                                                                                  │
│    446 │   # Check that the files exist (if not a URL).                                          │
│    447 │   for file in files:                                                                    │
│    448 │   │   if not file.startswith(("http", "www")) and not _os.path.isfile(file):            │
│ ❱  449 │   │   │   raise IOError("Missing input file: '%s'" % file)                              │
│    450 │                                                                                         │
│    451 │   # Try to read the files and return a molecular system.                                │
│    452 │   try:                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Missing input file: '../kpc2/equilibration/unbound//ligand_11/npt/ligand_11.prm7'

In [12]:
free_energy_protocol = bss.Protocol.FreeEnergy()
bss.FreeEnergy.Relative(unbound_system, free_energy_protocol, work_dir="/home/jguven/projects/metalloenzymes/scripts/")

In [13]:
bound_path = "../kpc2/equilibration/bound/"
ligand_1_number = 13
ligand_2_number = 6
system_1 = bss.IO.readMolecules([f"{bound_path}/system_{ligand_1_number}/npt/system_{ligand_1_number}.prm7",
                                        f"{bound_path}/system_{ligand_1_number}/npt/system_{ligand_1_number}.rst7"])
system_2 = bss.IO.readMolecules([f"{bound_path}/system_{ligand_2_number}/npt/system_{ligand_2_number}.prm7",
                                    f"{bound_path}/system_{ligand_2_number}/npt/system_{ligand_2_number}.rst7"])
system_1_ligand = None
protein = None
n_residues = [molecule.nResidues() for molecule in system_1]
n_atoms = [molecule.nAtoms() for molecule in system_1]
for j, (n_residues, n_atoms) in enumerate(zip(n_residues[:20], n_atoms[:20])):
    if n_residues == 1 and n_atoms > 5:
        system_1_ligand = system_1.getMolecule(j)
    elif n_residues > 1:
        protein = system_1.getMolecule(j)
    else:
        pass

system_2_ligand = None
n_residues = [molecule.nResidues() for molecule in system_2]
n_atoms = [molecule.nAtoms() for molecule in system_2]
for j, (n_residues, n_atoms) in enumerate(zip(n_residues, n_atoms)):
    if n_residues == 1 and n_atoms > 5:
        system_2_ligand = system_2.getMolecule(j)
    else:
        pass    

if system_1_ligand and system_2_ligand and protein:
    print(f"got molecules")
else:
    print("didn't work")

mapping = bss.Align.matchAtoms(system_1_ligand, system_2_ligand, complete_rings_only=True)
inverse_mapping = {value:key for key,value in mapping.items()}

system_2_ligand_aligned = bss.Align.rmsdAlign(system_2_ligand, system_1_ligand, inverse_mapping)

bound_merged_ligands = bss.Align.merge(system_1_ligand, system_2_ligand_aligned, mapping, allow_ring_breaking=True, allow_ring_size_change=True)

system_1.removeMolecules(system_1_ligand)
system_1.addMolecules(bound_merged_ligands)
bound_system = system_1

got molecules


In [14]:
bss.FreeEnergy.Relative(bound_system, free_energy_protocol, work_dir="/home/jguven/projects/metalloenzymes/scripts/")

In [6]:
second = mda.Universe("/home/jguven/projects/metalloenzymes/kpc2/inputs/protein/protein_complex.pdb")
first = mda.Universe("/home/jguven/projects/metalloenzymes/kpc2/inputs/protein/protein_1.pdb")
second_positions = second.select_atoms("protein").positions
first_positions = first.select_atoms("protein").positions
rmsd = MDAnalysis.analysis.rms.rmsd(second_positions, first_positions)
print(rmsd)

0.6327603865593754


In [3]:
full_path = os.getcwd() + "/"
if "scripts" in full_path:
    full_path = full_path.replace("/scripts/", "/")
equilibration_path = full_path + "/kpc2/" + "/equilibration/"
bound_path = equilibration_path + "bound/"
ligand_1_number = 1
ligand_2_number = 2
system_1_gmx = bss.IO.readMolecules([f"{bound_path}/system_{ligand_1_number}/npt/npt.gro",
                                     f"{bound_path}/system_{ligand_1_number}/npt/npt.top"])
system_2_gmx = bss.IO.readMolecules([f"{bound_path}/system_{ligand_2_number}/npt/npt.gro",
                                     f"{bound_path}/system_{ligand_2_number}/npt/npt.top"])


In [5]:
system_1_amber = bss.IO.saveMolecules(f"{bound_path}/system_{ligand_1_number}/ligand_{ligand_1_number}", system_1_gmx, ["PRM7", "RST7"])
system_2_amber = bss.IO.saveMolecules(f"{bound_path}/system_{ligand_2_number}/ligand_{ligand_2_number}", system_2_gmx, ["PRM7", "RST7"])
    

OSError: Failed to save system to format: 'RST7'

In [6]:
lig_14_system = mda.Universe("../kpc2/equilibration/bound/system_14/r_nvt/r_nvt.gro")
nv.show_mdanalysis(lig_14_system)

NGLWidget()

In [16]:
vim2_dry = bss.IO.readMolecules(["../vim_2/playground/vim2_dry.prmtop",
                                 "../vim_2/playground/vim2_dry.inpcrd"])[0]
print(vim2_dry.charge())     

print(vim2_dry.extract([232]))

-10.0000 |e|
<BioSimSpace.Molecule: nAtoms=1, nResidues=1>


In [45]:
universe = mda.Universe("../vim_2/playground/vim2_dry.prmtop",
                        "../vim_2/playground/vim2_dry.inpcrd")
system = universe.select_atoms("all")
ligand = universe.select_atoms("resname PT1")
print(f"ligand charge {ligand.total_charge()}")
active_site = pd.read_csv("../vim_2/inputs/protein/active_site.csv", header=None, names=["resname", "resid"])
active_site_resnames = active_site["resname"].tolist()

resnames = system.residues.resnames
resids = system.residues.resids
charges = []
other_charges = []
for i in range(len(resids)):
    if resnames[i] in active_site_resnames:
        residue = universe.select_atoms(f"resname {resnames[i]}")
        charge = residue.total_charge()
        print(f"residue {resnames[i]} {resids[i]} has charge {charge}")
        charges.append(float(charge))

    # if resnames[i] != "PT1" and resnames[i] != "WAT" and resnames[i] not in active_site_resnames:
    #     residue = universe.select_atoms(f"resname {resnames[i]}")
    #     charge = residue.total_charge()
    #     other_charges.append(float(charge))

net_charge = sum(charges)
print(net_charge)

not_ligand = universe.select_atoms("not resname PT1")
print(not_ligand.total_charge())


ligand charge -0.7114419266581535
residue HD1 83 has charge 0.054078073939308524
residue HE1 85 has charge 0.5690882261842489
residue AP1 87 has charge -0.3650490939617157
residue HD2 148 has charge 0.25752707151696086
residue CS1 167 has charge -0.31586899794638157
residue HD3 209 has charge 0.41295304521918297
residue ZN1 231 has charge 0.9915969967842102
residue ZN2 232 has charge 1.107118010520935
2.7114433322567493
-9.288554110797122


In [49]:
universe = mda.Universe("../md_vim2/outputs/vim2_large.pdb")
vim2_large = universe.select_atoms("all")
resids = vim2_large.residues.resids
resnames = vim2_large.residues.resnames
for i in range(len(resids)):
    print(f"{resids[i]} {resnames[i]}")

82 ACE
83 HID
84 GLY
85 HIE
86 GLY
87 ASP
88 NME
147 ACE
148 HID
149 NME
166 ACE
167 CYS
168 NME
208 ACE
209 HID
210 NME
231 ZN
232 ZN
233 P9T


0.0


In [4]:
ligand = bss.IO.readMolecules("../vim_2/inputs/ligands/docked_1.sdf")[0]
print(ligand.charge())

0.0000e+00 |e|


In [9]:
hpp = bss.IO.readMolecules("../vim_2/inputs/protein/vim2_hpp.pdb")[0]
print(hpp)
print(hpp.charge())

<BioSimSpace.Molecule: nAtoms=3456, nResidues=245>
0.0000e+00 |e|


In [11]:
pt1 = bss.IO.readMolecules("../vim_2/inputs/protein/PT1.mol2")[0]
print(pt1.charge())

-0.7114 |e|


In [10]:
protein = bss.IO.readMolecules(["../vim_2/inputs/protein/prot_water.prm7",
                                "../vim_2/inputs/protein/prot_water.rst7"])[0]
print(protein)                                
print(protein.charge())                                

<BioSimSpace.Molecule: nAtoms=3417, nResidues=232>
-9.2886 |e|


In [15]:
nv.show_file("../kpc2/equilibration/unbound/ligand_8/r_nvt/r_nvt.gro")

NGLWidget()

In [22]:
active_site = pd.read_csv("../vim_2/inputs/protein/active_site.csv", header=None, names=["resname", "resid"])
active_site_residues = active_site["resname"].tolist()
active_site_residue_ids = active_site["resid"].tolist()

universe = mda.Universe("../vim_2/inputs/protein/protein_complex.pdb")
atom_group = universe.select_atoms(f"resid {active_site_residue_ids[0]}")
atom_group.residues.resnames = active_site_residues[0]
atom_group.residues.resnames
universe.select_atoms(f"resid {active_site_residue_ids[0]}").resnames
universe.atoms.write("test.pdb")


In [13]:
import csv
with open('../vim_2/inputs/protein/active_site.csv', 'r') as file:
    reader = csv.readlines()


# returns ','

AttributeError: module 'csv' has no attribute 'readlines'

In [4]:
active_site = pd.read_csv("../vim_2/inputs/protein/active_site_copy.txt", header=None, names=["resname", "resid"])
active_site

,resname,resid
0,HD1 83,NaN
1,HE1 85,NaN
2,AP1 87,NaN
3,HD2 148,NaN
4,CS1 167,NaN
5,HD3 209,NaN


In [2]:
import glob
test = glob.glob("../kpc2/inputs/ligands/ligand_1_solvated*")
print(test)

['../kpc2/inputs/ligands/ligand_1_solvated.gro', '../kpc2/inputs/ligands/ligand_1_solvated.top']


In [2]:
free_directory = "/home/jguven/unbound/"
pmf_free, overlap_matrix_free = bss.FreeEnergy.Relative.analyse(free_directory)

pmf_bound, overlap_matrix_bound = bss.FreeEnergy.Relative.analyse(bound_directory)
free_energy_difference, free_energy_error = bss.FreeEnergy.Relative.difference(pmf_bound, pmf_free)

/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/alchemlyb/preprocessing/subsampling.py:135: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  return df.sort_index(0).reset_index('time').duplicated('time').any()
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/alchemlyb/preprocessing/subsampling.py:133: FutureWarning: In a future version of pandas all arguments of Series.sort_index will be keyword-only.
  return df.sort_index(0).reset_index('time', name='').duplicated('time').any()
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/alchemlyb/preprocessing/subsampling.py:135: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  return df.sort_index(0).reset_index('time').duplicated('time').any()
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/alchemlyb/preprocessing/subsampling.py:133: Future

In [3]:
pmf_free

[(0.0, 0.0000e+00 kcal/mol, 0.1263 kcal/mol),
 (0.125, 9.4915 kcal/mol, 0.0000e+00 kcal/mol),
 (0.25, 6.3567 kcal/mol, 0.1496 kcal/mol),
 (0.375, -1.9805 kcal/mol, 0.1575 kcal/mol),
 (0.5, -11.7766 kcal/mol, 0.1598 kcal/mol),
 (0.625, -6.9513 kcal/mol, 0.1647 kcal/mol),
 (0.75, 16.9964 kcal/mol, 0.7148 kcal/mol),
 (0.875, 34.2116 kcal/mol, 0.6545 kcal/mol),
 (1.0, 42.5350 kcal/mol, 0.6464 kcal/mol)]

In [4]:
overlap_matrix_free

matrix([[9.97193833e-01, 2.71904833e-03, 1.14636118e-05, 1.75632486e-12,
         5.28367192e-15, 2.34408067e-17, 1.13940050e-12, 1.15846327e-08,
         7.56432246e-05],
        [8.92222584e-03, 9.84741734e-01, 6.33547540e-03, 4.02486897e-07,
         6.14447531e-09, 7.07260002e-10, 9.73846692e-13, 2.37954497e-09,
         1.53059092e-07],
        [2.31939541e-05, 3.90639581e-03, 9.59299807e-01, 3.62058081e-02,
         3.39304264e-04, 1.49291498e-04, 6.18717281e-05, 1.38934843e-05,
         4.33852762e-07],
        [3.81602359e-12, 2.66502731e-07, 3.88804360e-02, 8.17854822e-01,
         1.37151472e-01, 6.11298115e-03, 2.23662098e-08, 9.05586163e-13,
         1.31654640e-13],
        [5.28367192e-15, 1.87252885e-09, 1.67701132e-04, 6.31239650e-02,
         9.14384656e-01, 2.23236760e-02, 1.99819867e-13, 1.36207282e-21,
         7.03771152e-23],
        [2.34408067e-17, 2.15537486e-10, 7.37873228e-05, 2.81349957e-03,
         2.23236760e-02, 9.74789036e-01, 9.02619530e-10, 2.10700472

In [13]:
import numpy as np
n_windows = 11
lambda_array = list(np.around(np.linspace(0, 1, int(n_windows)), decimals=5))
lambda_list = [str(item) for item in lambda_array]

lambda_array_bash = " ".join(lambda_list)
with open("test.dat", "w") as file:
    file.write(lambda_array_bash)

In [3]:
columns = ["ATOM", "atom_id", "name", "resname", "resid", "X", "Y", "Z", "num", "zero", "element"]
pdb = pd.read_csv("../vim_2/inputs/protein/protein_complex.pdb", sep="\s+", skiprows=1, header=None, dtype=str, names=columns).dropna()
print(pdb.head())

pd_to_list = lambda column: pdb[column].tolist()
atoms = pd_to_list("ATOM")
atom_ids = pd_to_list("atom_id")
atom_names = pd_to_list("name")
residue_names = pd_to_list("resname")
residue_ids = pd_to_list("resid")
x, y, z = pd_to_list("X"), pd_to_list("Y"), pd_to_list("Z")
numbers = pd_to_list("num")
zeros = pd_to_list("zero")
elements = pd_to_list("element")

   ATOM atom_id name resname resid        X       Y       Z   num  zero  \
0  ATOM       1    N     GLU     1  -16.230  -5.653  12.548  0.00  0.00   
1  ATOM       2   H1     GLU     1  -15.787  -6.381  12.805  0.23  0.00   
2  ATOM       3   H2     GLU     1  -17.036  -5.649  12.925  0.23  0.00   
3  ATOM       4   H3     GLU     1  -16.317  -5.657  11.662  0.23  0.00   
4  ATOM       5   CA     GLU     1  -15.494  -4.464  12.960  0.05  0.00   

  element  
0       N  
1       H  
2       H  
3       H  
4       C  



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



In [4]:
active_site_df = pd.read_csv("../vim_2/inputs/protein/active_site.csv", header=None, names=["resname", "resid"], dtype=str)
# print(active_site_df.head())

active_site_resnames = active_site_df["resname"].tolist()
active_site_resids = active_site_df["resid"].tolist()

for i in range(len(active_site_resnames)):
    for j in range(len(residue_ids)):
        if residue_ids[j] == active_site_resids[i].strip():
            residue_names[j] = active_site_resnames[i]
            print(residue_names[j])

HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HD1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
HE1
AP1
AP1
AP1
AP1
AP1
AP1
AP1
AP1
AP1
AP1
AP1
AP1
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
HD2
CS1
CS1
CS1
CS1
CS1
CS1
CS1
CS1
CS1
CS1
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3
HD3


In [31]:
complex_file = "../vim_2/inputs/protein/protein_complex.pdb"

with open("test.pdb", "w") as pdb:
    for i in range(len(residue_names)):
        pdb.write(f"{atoms[i]:10}{atom_ids[i]:4}{atom_names[i]:4}{residue_names[i]:8}{residue_ids[i]:5}{x[i]:9}{y[i]}  {z[i]:9}\n")


In [48]:
complex_file = "../vim_2/inputs/protein/protein_complex.pdb"
with open(complex_file, "r") as pdb:
    lines = pdb.readlines()

n_zn = 0
for line in lines:
    if "ZN" in line:
        n_zn += 1


protein_terminus = [i for i in range(len(lines)) if "ZN" in lines[i]][0] - 1
zn_line = protein_terminus+1
ids = []
for i in range(len(lines[zn_line])):
    c = lines[zn_line][i]
    if c == "Z":
        ids.append(i)

test = lines[zn_line]
print(test[ids[1]], test[ids[1]+1], test[ids[1]+2])

zn_terminus = protein_terminus + n_zn + 1
conect = [i for i in range(len(lines)) if "CONECT" in lines[i]][0]
with open("test.pdb", "w") as test:
    atom = "ATOM"
    test.write(f"{atom:10}1")

Z N  


In [4]:
!head -n 10 ../vim_2/inputs/ligands/docked_8.pdb

HEADER                                                                          
TITLE     A P9T 403                                                             
HETATM    1  C02 P9T A 403     -34.220   3.393   8.083  1.00 30.69           C  
HETATM    2  C03 P9T A 403     -33.874   4.740   8.102  1.00 27.72           C  
HETATM    3  C04 P9T A 403     -34.884   5.693   8.127  1.00 23.83           C  
HETATM    4  C05 P9T A 403     -34.646   7.058   7.483  1.00 20.13           C  
HETATM    5  C10 P9T A 403     -36.192   5.293   8.463  1.00 29.34           C  
HETATM    6  C11 P9T A 403     -37.204   6.225   8.768  1.00 33.81           C  
HETATM    7  C13 P9T A 403     -38.480   5.782   9.094  1.00 33.91           C  
HETATM    8  C14 P9T A 403     -38.759   4.419   9.121  1.00 35.18           C  


In [10]:
with open("../kpc2/inputs/ligands/docked_8_fixed.pdb", "r") as pdb:
    lines = pdb.readlines()
    new_lines = [line+"\n" for line in lines]
print(new_lines)

['HEADER                                                                          \n\n', 'TITLE     TWB_D (1)                                                             \n\n', 'HETATM    1  C02   MOL     1      22.990  15.325   9.444  1.00  0.00           C  \n\n', 'HETATM    2  C03   MOL     1      21.947  16.051  10.102  1.00  0.00           C  \n\n', 'HETATM    3  C04   MOL     1      21.342  17.076   9.469  1.00  0.00           C  \n\n', 'HETATM    4  C05   MOL     1      20.231  17.792  10.209  1.00  0.00           C  \n\n', 'HETATM    5  C10   MOL     1      21.763  17.377   8.100  1.00  0.00           C  \n\n', 'HETATM    6  C11   MOL     1      21.178  18.394   7.333  1.00  0.00           C  \n\n', 'HETATM    7  C13   MOL     1      21.594  18.632   6.052  1.00  0.00           C  \n\n', 'HETATM    8  C14   MOL     1      22.605  17.867   5.495  1.00  0.00           C  \n\n', 'HETATM    9  C16   MOL     1      23.189  16.871   6.232  1.00  0.00           C \n\n', 'HETATM   10  

In [6]:
lig = mda.Universe("../kpc2/inputs/ligands/docked_8_fixed.pdb")
# atoms = lig.atoms
# for i in range(len(atoms)):
#     print(atoms[i].id, atoms[i].name)
view = nv.show_mdanalysis(lig)
view

ValueError: Failed to construct topology from file ../kpc2/inputs/ligands/docked_8_fixed.pdb with parser <class 'MDAnalysis.topology.PDBParser.PDBParser'>.
Error: Formal charge C is unrecognized

In [14]:
view.add

NGLWidget()

In [ ]:
lig = mda.Universe("../vim_2/inputs/ligands/docked_8.pdb")
nv.show_mdanalysis(lig)

NGLWidget()

In [3]:
lig = mda.Universe("../vim_2/inputs/ligands/docked_8.pdb")
nv.show_mdanalysis(lig)

NGLWidget()

In [2]:
u = mda.Universe("../inputs/protein/vim2.gromacs.pdb")
lig = u.select_atoms("resname PT1")
v = nv.show_mdanalysis(lig)
v

NGLWidget()

In [121]:
with open("../../md_vim2/outputs/parmed/pt1_dry.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_pt1 = df["Charge"].astype(float).sum()


-0.7112999999999998


In [112]:
with open("../../md_vim2/outputs/pt1_parmed.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_pt1 = df["Charge"].astype(float).sum()
# net_charge_pt1
print(df)

    ATOM  RES RESNAME  NAME TYPE At.#  Radius   Depth     Mass   Charge  \
0   3418  233     PT1   C02    c    6  1.8606  0.0988  12.0100   0.8294   
1   3419  233     PT1   C03   cc    6  1.8606  0.0988  12.0100  -0.4941   
2   3420  233     PT1   C04   cd    6  1.8606  0.0988  12.0100   0.2997   
3   3421  233     PT1   C05   c3    6  1.9069  0.1078  12.0100  -0.3215   
4   3422  233     PT1   C10   ca    6  1.8606  0.0988  12.0100  -0.3040   
5   3423  233     PT1   C11   ca    6  1.8606  0.0988  12.0100   0.2951   
6   3424  233     PT1   C13   ca    6  1.8606  0.0988  12.0100  -0.4058   
7   3425  233     PT1   C14   ca    6  1.8606  0.0988  12.0100   0.4017   
8   3426  233     PT1   C16   ca    6  1.8606  0.0988  12.0100  -0.4571   
9   3427  233     PT1   C17   ca    6  1.8606  0.0988  12.0100   0.4785   
10  3428  233     PT1   F12    f    9  1.7029  0.0832  19.0000  -0.1145   
11  3429  233     PT1   F15    f    9  1.7029  0.0832  19.0000  -0.1347   
12  3430  233     PT1   O

In [97]:
with open("../../md_vim2/outputs/cs1.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_cs1 = df["Charge"].astype(float).sum()
net_charge_cs1

-0.3157999999999999

In [106]:
zn1 = 0.9916
zn2 = 1.1071

-1.0270999999999997

In [99]:
with open("../../md_vim2/outputs/ap1.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_ap1 = df["Charge"].astype(float).sum()
net_charge_ap1

-0.36529999999999974

In [103]:
with open("../../md_vim2/outputs/hd3.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_hd3 = df["Charge"].astype(float).sum()
net_charge_hd3

0.41290000000000016

In [111]:
net_charge_cs1 + net_charge_ap1 + net_charge_hd3 + net_charge_pt1

-0.9794999999999993

In [113]:
net_charge_pt1 + zn1 + zn2

1.3874000000000002

In [114]:
net_charge_pt1

-0.7112999999999998

In [122]:
pt = bss.IO.readMolecules("../inputs/ligands/ligand_8_xtal.sdf")
pt.charge()

0.0000e+00 |e|

In [153]:
parm_u = mda.Universe("../inputs/protein/PT1.mol2")
xtal_u = mda.Universe("../inputs/ligands/ligand_8_xtal.mol2")
new_coords = xtal_u.atoms.positions
new_univers = parm_u.load_new(new_coords)
parm_u.atoms.positions

array([[25.556 , 27.158 , 39.628 ],
       [25.902 , 28.505 , 39.647 ],
       [24.892 , 29.458 , 39.672 ],
       [25.13  , 30.823 , 39.028 ],
       [23.584 , 29.058 , 40.008 ],
       [22.572 , 29.99  , 40.313 ],
       [21.296 , 29.547 , 40.639 ],
       [21.017 , 28.184 , 40.666 ],
       [22.013 , 27.265 , 40.366 ],
       [23.307 , 27.712 , 40.035 ],
       [22.84  , 31.327 , 40.288 ],
       [19.763 , 27.751 , 40.986 ],
       [26.4   , 26.334 , 39.508 ],
       [25.14  , 32.112 , 36.5881],
       [26.306 , 29.941 , 36.692 ],
       [23.842 , 30.02  , 36.724 ],
       [24.268 , 26.785 , 39.745 ],
       [25.103 , 30.719 , 37.17  ],
       [24.416 , 31.455 , 39.33  ],
       [26.02  , 31.172 , 39.322 ],
       [26.863 , 28.783 , 39.643 ],
       [20.577 , 30.208 , 40.854 ],
       [21.812 , 26.286 , 40.385 ],
       [23.031 , 30.585 , 36.911 ]], dtype=float32)

In [154]:
new_univers.atoms.positions

array([[25.556 , 27.158 , 39.628 ],
       [25.902 , 28.505 , 39.647 ],
       [24.892 , 29.458 , 39.672 ],
       [25.13  , 30.823 , 39.028 ],
       [23.584 , 29.058 , 40.008 ],
       [22.572 , 29.99  , 40.313 ],
       [21.296 , 29.547 , 40.639 ],
       [21.017 , 28.184 , 40.666 ],
       [22.013 , 27.265 , 40.366 ],
       [23.307 , 27.712 , 40.035 ],
       [22.84  , 31.327 , 40.288 ],
       [19.763 , 27.751 , 40.986 ],
       [26.4   , 26.334 , 39.508 ],
       [25.14  , 32.112 , 36.5881],
       [26.306 , 29.941 , 36.692 ],
       [23.842 , 30.02  , 36.724 ],
       [24.268 , 26.785 , 39.745 ],
       [25.103 , 30.719 , 37.17  ],
       [24.416 , 31.455 , 39.33  ],
       [26.02  , 31.172 , 39.322 ],
       [26.863 , 28.783 , 39.643 ],
       [20.577 , 30.208 , 40.854 ],
       [21.812 , 26.286 , 40.385 ],
       [23.031 , 30.585 , 36.911 ]], dtype=float32)

< Timestep 0 >


In [14]:
with open("../kpc2/inputs/ligands/docked_1.mol2", "r") as file:
    lines = file.readlines()

atom_idx = [index for index in range(len(lines)) if "ATOM" in lines[index]][0]
bond_idx = [index for index in range(len(lines)) if "BOND" in lines[index]][0]

new_lines = lines[atom_idx+1:bond_idx]
charges  = [float(line.split()[-1]) for line in new_lines]
net_charge = sum(charges)
print(net_charge)
# info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
# column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
# df = pd.DataFrame(info[1:], columns=column_headers)
# net_charge_pt1 = df["Charge"].astype(float).sum()

-0.12750000000000006
